# Python数据库sqlite3基本用法

## 1.创建和写入数据库

假设我们需要将一些"task"信息存入本地数据库, 其中包含"task_id", "task_name", "status", “priority”, "finish_time"条目，并实现其存储、读取、查找、修改、删除等基本操作。``python3``标准库中的``sqlite3``可以方便地做到。

In [1]:
import sqlite3

In [2]:
# 连接到当前路径下的 SQLite 数据库（如果不存在，则会创建一个新的数据库文件）
conn = sqlite3.connect("tasks.db")

# 创建一个游标对象，用于执行 SQL 查询
cursor = conn.cursor()

# 创建一个名为 "tasks" 的表格
cursor.execute('''
    CREATE TABLE IF NOT EXISTS tasks (
        task_id INTEGER PRIMARY KEY,
        task_name TEXT,
        status TEXT DEFAULT '未完成',
        priority INTEGER NOT NULL,
        finish_time TIMESTAMP
    )
''')

上面采用``[表头] [数据文本类型] [约束](可选)``形式定义表格的一列。例如``PRIMARY KEY``要求列值必须是唯一且非空的，用于唯一标识表格中的每一行数据。

In [3]:
# 插入一些示例任务数据
cursor.execute("INSERT INTO tasks (task_name, status, priority, finish_time) VALUES (?, ?, ?, ?)",
               ("完成项目报告", "未完成", 2, "2023-08-31 15:00:00"))
cursor.execute("INSERT INTO tasks (task_name, status, priority, finish_time) VALUES (?, ?, ?, ?)",
               ("准备会议材料", "已完成", 1, "2023-09-05 10:30:00"))
cursor.execute("INSERT INTO tasks (task_name, status, priority, finish_time) VALUES (?, ?, ?, ?)",
               ("清理办公室", "未完成", 3, "2023-09-10 14:45:00"))

# 提交更改并关闭连接
conn.commit()
conn.close()

这里时间戳``TIMESTAMP``采用ISO 8601 格式。标识数据``task_id``已经自动生成（见下文），自然也可以按照上面语法手动赋值。

## 2.读取和查找

为了方便演示，下面定义打印任务数据的函数

In [4]:
def print_tasks(tasks):
    for task in tasks:
        task_id, task_name, status, priority, finish_time = task
        print(f"Task ID: {task_id}")
        print(f"Task Name: {task_name}")
        print(f"Status: {status}")
        print(f"Priority: {priority}")
        print(f"Finish Time: {finish_time}")
        print("------------------------")

In [13]:
# 再次连接数据库和创建游标
conn = sqlite3.connect("tasks.db")
cursor = conn.cursor()

# 执行 SQL 查询以检索任务数据
cursor.execute("SELECT * FROM tasks")

# 获取所有任务数据
tasks = cursor.fetchall()

# 打印任务数据
print_tasks(tasks)

Task ID: 1
Task Name: 完成项目报告
Status: 未完成
Priority: 2
Finish Time: 2023-08-31 15:00:00
------------------------
Task ID: 2
Task Name: 准备会议材料
Status: 已完成
Priority: 1
Finish Time: 2023-09-05 10:30:00
------------------------
Task ID: 3
Task Name: 清理办公室
Status: 未完成
Priority: 3
Finish Time: 2023-09-10 14:45:00
------------------------


执行对特定条目的查找，也即更具体地描述SQL语法的``SELECT``语句：

In [14]:
# 执行 SQL 查询以检索特定 task_name 的任务数据
cursor.execute("SELECT * FROM tasks WHERE task_name=?", ("清理办公室",))
print_tasks(cursor.fetchall())

Task ID: 3
Task Name: 清理办公室
Status: 未完成
Priority: 3
Finish Time: 2023-09-10 14:45:00
------------------------


若相应字段为序列，可对特定的区间查找：

In [15]:
cursor.execute("SELECT * FROM tasks WHERE finish_time BETWEEN ? AND ?", ("2023-09-00 14:45:00", "2023-09-10 14:45:00"))
print_tasks(cursor.fetchall())

Task ID: 2
Task Name: 准备会议材料
Status: 已完成
Priority: 1
Finish Time: 2023-09-05 10:30:00
------------------------
Task ID: 3
Task Name: 清理办公室
Status: 未完成
Priority: 3
Finish Time: 2023-09-10 14:45:00
------------------------


## 3.更新和删除

删除的语法和查找完全类似

In [16]:
cursor.execute("DELETE FROM tasks WHERE task_id BETWEEN ? AND ?", (1, 2))

更新则只需在此语法结构上使用``SET``:

In [17]:
new_status = "已完成"
cursor.execute("UPDATE tasks SET status=? WHERE task_id=?", (new_status, 3))

再次查看数据库中表格内容：

In [18]:
cursor.execute("SELECT * FROM tasks")
print_tasks(cursor.fetchall())

Task ID: 3
Task Name: 清理办公室
Status: 已完成
Priority: 3
Finish Time: 2023-09-10 14:45:00
------------------------


此时本地会产生一个``-journal``临时文件，直至.commit()才会对数据库进行修改，同``git``颇有相似之处。

In [19]:
# 放弃修改则直接关闭
# conn.close()
conn.commit()

## 4.注记

``sqilite3``的异常处理是比较完备的，当发生输入数据类型错误、标识重复、链接已关闭等情形，均已定义了可读性良好的异常类型。

In [20]:
# 关闭游标和连接
cursor.close()
conn.close()

cursor.execute("INSERT INTO tasks (task_name, status, priority, finish_time) VALUES (?, ?, ?, ?)",
               ("下班", "未完成", 0, "2023-08-25 17:25:00"))

ProgrammingError: Cannot operate on a closed cursor.